In [0]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import copy
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, sampler
import torchvision
import torchvision.transforms as transforms

In [0]:
# skip these lines if not using Google Colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# skip this line if not using Google Colab
cd 'drive/My Drive/Colab Notebooks/small projects/CIFAR10_PyTorch'

/content/drive/My Drive/Colab Notebooks/small projects/CIFAR10_PyTorch


In [0]:
# create DataLoader class for training set
transform = transforms.Compose([
    transforms.ToTensor()
])
dataset = torchvision.datasets.CIFAR10('datasets', train=True, transform=transform)
loader = DataLoader(dataset, batch_size=64)

In [0]:
# calculate statistical information of training set
def calculate_mean_std(loader):
  """
  Calculate mean and std over 3 channels of training data
  
  Input: dataloader for training set
  
  Return tuple of mean and std in type Tensor
  """
  mean, std = 0.0, 0.0
  num_samples = 0
  for x, y in loader:
    batch_size = x.size(0)
    x = x.view(batch_size, x.size(1), -1)
    mean += torch.mean(x, -1).sum(0)
    std += torch.std(x, -1).sum(0)
    num_samples += batch_size
  return mean/num_samples, std/num_samples

In [0]:
mean_train, std_train = calculate_mean_std(loader)
print('Mean: ', mean_train, 'Std: ', std_train)

Mean:  tensor([0.4914, 0.4822, 0.4465]) Std:  tensor([0.2023, 0.1994, 0.2010])


In [0]:
# data augmentation and normalization for training data
transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean_train, std_train)
])

# only normalization for validation and test data
transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean_train, std_train)
])

In [0]:
# split original training set into 49000 train data and 1000 validation data
NUM_TRAIN = 49000
NUM_ALL = 50000

cifar_train = torchvision.datasets.CIFAR10('datasets', train=True, transform=transform_train)
cifar_val = torchvision.datasets.CIFAR10('datasets', train=True, transform=transform_test)
cifar_test = torchvision.datasets.CIFAR10('datasets', train=False, transform=transform_test)

train_set = DataLoader(cifar_train, batch_size=64, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))
val_set = DataLoader(cifar_val, batch_size=64, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, NUM_ALL)))
test_set = DataLoader(cifar_test, batch_size=64)

In [0]:
# use GPU if available
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [0]:
def validate(model, dataset, criterion):
  """
  Check accuracy and loss of given dataset
  """
  
  model = model.to(device)
  model.eval()  # set model to evaluation mode
  running_loss = 0
  num_correct = 0
  num_samples = 0
  batches = 0
  with torch.no_grad():
    for t, (X, y) in enumerate(dataset):
      batches += 1
      X = X.to(device, dtype=torch.float32)
      y = y.to(device, dtype=torch.long)
      output = model(X)
      loss = criterion(output, y)
      preds = torch.max(output, 1)[1]
      
      running_loss += loss.item()
      num_correct += (preds == y).sum()
      num_samples += preds.size(0)
    model.train()
    acc = 100. * float(num_correct) / float(num_samples)
    running_loss /= batches
    return acc, running_loss

def train(model, criterion, optimizer, scheduler, epochs=1, validate_every=1):
  """
  Train and return best model based on validation accuracy
  """
  
  best_acc = 0.0
  model.to(device)
  model.train()  # set model to training mode
  for e in range(epochs):
    num_samples = 0
    scheduler.step()
    running_loss = 0
    
    print('Epoch %d' %e)
    print('-------')
    for t, (X, y) in enumerate(train_set):
      X = X.to(device, dtype=torch.float32)
      y = y.to(device, dtype=torch.long)
      optimizer.zero_grad()
      output = model(X)
      preds = torch.max(output, 1)[1]
      loss = criterion(output, y)
      loss.backward()
      optimizer.step()
      
      running_loss += loss.item()
      corrects = (preds == y).sum()
      num_samples += X.size(0)
      if t % validate_every == 0:
        acc_val, loss_val = validate(model, val_set, criterion)
        print('Iteration %d, training loss = %.3f, validation loss = %.3f, val acc = %.3f' %(t, running_loss/(t+1), loss_val, acc_val))
        if acc_val > best_acc:
          best_acc = acc_val
          torch.save(model.state_dict(), 'best_model.pth')
    acc_val, loss_val = validate(model, val_set, criterion)
    if acc_val > best_acc:
      best_acc = acc_val
      torch.save(model.state_dict(), 'best_model.pth')
    print()
  model.load_state_dict(torch.load('best_model.pth'))
  return model

In [0]:
model = torchvision.models.resnet34(pretrained=True)

#  freeze first layers
for param in model.parameters():
  param.requires_grad = False
  
#  modify FC layer to have the same output as our dataset (10 classes), requires_grad = True by default
model.fc = nn.Linear(model.fc.in_features, 10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

#  decrease learning rate every number of steps
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

In [0]:
# run this line if you want to retrain, otherwise skip to next line
model = train(model, criterion, optimizer, lr_scheduler, epochs=5, validate_every=100)

Epoch 0
-------
Iteration 0, training loss = 2.450, validation loss = 2.808, val acc = 14.400
Iteration 100, training loss = 0.896, validation loss = 0.495, val acc = 83.100
Iteration 200, training loss = 0.734, validation loss = 0.435, val acc = 86.000
Iteration 300, training loss = 0.680, validation loss = 0.423, val acc = 85.800
Iteration 400, training loss = 0.644, validation loss = 0.416, val acc = 86.100
Iteration 500, training loss = 0.621, validation loss = 0.414, val acc = 87.000
Iteration 600, training loss = 0.610, validation loss = 0.438, val acc = 86.300
Iteration 700, training loss = 0.595, validation loss = 0.448, val acc = 86.100

Epoch 1
-------
Iteration 0, training loss = 1.184, validation loss = 0.400, val acc = 87.200
Iteration 100, training loss = 0.473, validation loss = 0.382, val acc = 88.200
Iteration 200, training loss = 0.471, validation loss = 0.381, val acc = 87.200
Iteration 300, training loss = 0.467, validation loss = 0.365, val acc = 88.400
Iteration 4

In [0]:
# run this line if you want to use pre-trained weights
model.load_state_dict(torch.load('best_model.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
# some validation of our model
acc_train, _ = validate(model, train_set, criterion)
acc_val, _ = validate(model, val_set, criterion)
acc_test, _ = validate(model, test_set, criterion)

print('Train accuracy: %.3f%%' %acc_train)
print('Validation accuracy: %.3f%%' %acc_val)
print('Test accuracy: %.3f%%' %acc_test)

Train accuracy: 85.704%
Validation accuracy: 90.500%
Test accuracy: 88.050%
